In [1]:
import numpy as np
import glob as glob
from os.path import join, basename, dirname, exists
import sys
print(sys.executable)  #print kernel path
#print(sys.path)

# tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras import layers
print(tf.__version__)

# custom imports
import librispect as lspct
from librispect.features import predict
from librispect.utils import split_validation

# cpc import
sys.path.insert(1, '../')
from data_utils import SortedNumberGenerator
sys.path.insert(1, 'notebooks/')

/home/AD/kachiem/miniconda3/envs/tf-conda/bin/python3
1.13.1


attempting to connect srihita's memmap spectrogram function to train_model  


In [2]:
'''
Instead of images and labels, I have spectrogram segments x and y.
So x is input and y would be output (aka labels). 
You train on bunch of (x - power,y - number of samples), let's call it train_x and train_y.
Then when you input test_x, you should get test_y (or vis_x and vis_y in this case)
'''

# spect_height = number of freq. bins
# y always has one time bin
# n_lags is number of time bins of x
# shape of x = (n_lags,spect_height) x number of samples
# shape of y = (1, spect_height) x number of samples

n_lags = 2
spect_height = 32
NIN = n_lags*spect_height

# args for the following class, MemmapHandler
datafolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus/'
visfolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/'
n_lags = 2
spect_height = 32
#subset = 'train'

In [87]:
# convert to array
def memmap_to_arr(memmap, shape):
    # memmap: input np.memmap()
    # shape: (samples, channels, features (freq))
    
    memmap = np.asarray(memmap.T[:].tolist()).reshape(shape)
    lst = [np.asarray(memmap[i]) for i in range(0,len(memmap))]
    arr = np.array(lst)
    print(arr.shape)
    
    return arr

In [79]:
x_data = memmap_to_arr(xdata, (xdata.shape[1], 2, 32))

(149962, 2, 32)
(149962, 2, 32)


train data

In [12]:
# loading train data
why = np.lib.format.open_memmap('y.npy', dtype='float32', mode='r')
xdata = np.lib.format.open_memmap('x_data.npy', dtype='float32', mode='r', shape=(n_lags*spect_height, num_data_samples))
ydata = np.lib.format.open_memmap('y_data.npy', dtype='float32', mode='r', shape=(spect_height, num_data_samples))

num_data_samples = int(len(why) / spect_height)

why.shape, xdata.shape, ydata.shape

((4798784,), (64, 149962), (32, 149962))

In [85]:
data = why[:]
x_data = memmap_to_arr(xdata, (xdata.shape[1], 2, 32))
y_data = memmap_to_arr(ydata, (ydata.shape[1], 1, 32))

data.shape

(149962, 2, 32)
(149962, 1, 32)


(4798784,)

In [32]:
# subsetting to match vis data
x_data = x_data[:16653, :, :]
y_data = y_data[:16653, :, :]

x_data.shape, y_data.shape

((16653, 2, 32), (16653, 1, 32))

test data

In [46]:
# loading test data
why_vis = np.lib.format.open_memmap('y_vis.npy', dtype='float32', mode='r')
xdata_vis = np.lib.format.open_memmap('x_data_vis.npy', dtype='float32', mode='r', shape=(n_lags*spect_height, num_data_samples_vis))
ydata_vis = np.lib.format.open_memmap('y_data_vis.npy', dtype='float32', mode='r', shape=(spect_height, num_data_samples_vis))

num_data_samples_vis = int(len(why_vis) / spect_height)

why_vis.shape, xdata_vis.shape, ydata_vis.shape

((32, 16653), (64, 16653), (32, 16653))

In [47]:
data_vis = why_vis[:]
x_data_vis = memmap_to_arr(xdata_vis, (xdata_vis.shape[1], 2, 32))
y_data_vis = memmap_to_arr(ydata_vis, (ydata_vis.shape[1], 1, 32))

data_vis.shape

(16653, 2, 32)
(16653, 1, 32)


(32, 16653)

data (arrays) into iterator class

In [81]:
class Shuffled_memmap_subset(object):
    
    ''' For creating train/valid/test shuffled memmap dataset '''
    
    def __init__(self, data, x_data, y_data, subset, n_lags, spect_height, vis_ratio=0, valid_ratio=.06):
        
        # Set params
        self.data = data
        self.x_data = x_data
        self.y_data = y_data
        self.subset = subset
        self.n_lags = n_lags
        self.spect_height = spect_height
        self.num_data_samples = int(len(data) / spect_height)
        #self.num_data_samples

        '''
        # Initialize memmap handler
        self.memmap_handler = MemmapHandler(self.folder, self.n_lags, self.spect_height, self.subset)

        # Initialize data
        data_loc = self.memmap_handler.read_file_loc(self.folder)
        data = self.memmap_handler.init_data(self.folder, self.saved_data, x_location)
        self.num_data_samples = self.memmap_handler.get_n_samples('train')
        self.dataset = data.memmap_copy(data, data.shape[0], data.shape[1][:1000])
        '''
        
        
        # get dataset+batching indicies
        #self.num_vis_samples = int(self.num_data_samples*vis_ratio)
        self.num_valid_samples = int(self.num_data_samples*valid_ratio)
        #vis_start = np.random.randint(self.num_data_samples - self.num_vis_samples)
       
        #self.vis_idxs = np.arange(vis_start, vis_start + self.num_vis_samples)
        #self.data_idxs = np.delete(np.arange(self.num_data_samples), self.vis_idxs)
        self.data_idxs = np.arange(self.num_data_samples)
        #self.num_data_samples -= self.num_vis_samples
        self.n_batches = self.get_n_batches()
       
        np.random.shuffle(self.data_idxs)
        self.valid_idxs, self.data_idxs = np.split(self.data_idxs, [self.num_valid_samples])
        self.num_data_samples -= self.num_valid_samples
    
    def __len__(self):
        return self.n_batches
    
    def get_n_batches(self, batch_size=64):
        n_batches = 0
        for batch_idx in range(0, self.num_data_samples, batch_size):
            n_batches +=1
        return n_batches
            
    def get_num_samples(self, subset):
        ''' 
        Return number of samples wrt subset. 
            subset: a string - 'train', 'valid', or 'test'
        '''
        if subset == 'train':
            return self.num_data_samples
        elif subset == 'valid':
            return self.num_valid_samples
        elif subset == 'test':
            return self.num_data_samples_vis
    
    def get_data(self, idx):
        return self.x_data[:, :idx].T, self.y_data[idx].T
   
    def data_iterator(self, batch_size=64):
        np.random.shuffle(self.data_idxs)
        for batch_idx in range(0, self.num_data_samples, batch_size):
            shuff_idx = self.data_idxs[batch_idx:batch_idx+batch_size]
            
            yield self.get_data(shuff_idx)
   
    def valid_set(self):
        return self.get_data(self.valid_idxs)
   
    def valid_iterator(self, batch_size=64):
        for batch_idx in range(0, self.num_valid_samples, batch_size):
            valid_idx = self.valid_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(valid_idx)
   

In [82]:
# test iterator object
train_data = Shuffled_memmap_subset(
    data=data,
    x_data=x_data,
    y_data=y_data,
    subset='train', 
    n_lags=2,
    spect_height=32
    )

In [49]:
def network_encoder(x, code_size):

    ''' Define the network mapping images to embeddings '''
    print(x.shape)
    x = keras.layers.Conv1D(filters=2, kernel_size=2, strides=1, data_format="channels_last", activation='relu', input_shape=(32, 2))(x)
    print("conv1d", x.shape)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=2, kernel_size=2, strides=1, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=4, kernel_size=2, strides=2, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv1D(filters=4, kernel_size=2, strides=2, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Dense(units=code_size, activation='relu', name='encoder_embedding')(x)

    return x


def network_autoregressive(x):

    ''' Define the network that integrates information along the sequence '''

    # x = keras.layers.GRU(units=256, return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    x = keras.layers.GRU(units=256, return_sequences=False, name='ar_context')(x)

    return x


def network_prediction(context, code_size, predict_terms):

    ''' Define the network mapping context to multiple embeddings '''

    outputs = []
    for i in range(predict_terms):
        outputs.append(keras.layers.Dense(units=code_size, activation="linear", name='z_t_{i}'.format(i=i))(context))

    if len(outputs) == 1:
        output = keras.layers.Lambda(lambda x: K.expand_dims(x, axis=1))(outputs[0])
    else:
        output = keras.layers.Lambda(lambda x: K.stack(x, axis=1))(outputs)

    return output

class CPCLayer(keras.layers.Layer):

    ''' Computes dot product between true and predicted embedding vectors '''

    def __init__(self, **kwargs):
        super(CPCLayer, self).__init__(**kwargs)

    def call(self, inputs):

        # Compute dot product among vectors
        preds, y_encoded = inputs
        dot_product = K.mean(y_encoded * preds, axis=-1)
        dot_product = K.mean(dot_product, axis=-1, keepdims=True)  # average along the temporal dimension

        # Keras loss functions take probabilities
        dot_product_probs = K.sigmoid(dot_product)

        return dot_product_probs

    def compute_output_shape(self, input_shape):
        #return (input_shape[0][0],1)
        return (input_shape[0], 1)

def network_cpc(spect_height, n_lags, terms, predict_terms, code_size, input_shape, learning_rate):

    ''' Define the CPC network combining encoder and autoregressive model '''

    # Set learning phase (https://stackoverflow.com/questions/42969779/keras-error-you-must-feed-a-value-for-placeholder-tensor-bidirectional-1-keras)
    K.set_learning_phase(1)

    # Define encoder model
    encoder_input = keras.layers.Input(shape=(spect_height, n_lags))
    #print(encoder_input.shape)
    encoder_output = network_encoder(encoder_input, code_size)
    #print(encoder_output.shape)
    encoder_model = keras.models.Model(encoder_input, encoder_output, name='encoder')
    encoder_model.summary()

    # Define rest of model
    x_input = keras.layers.Input((terms, spect_height, n_lags))
    x_encoded = keras.layers.TimeDistributed(encoder_model)(x_input)
    context = network_autoregressive(x_encoded)
    preds = network_prediction(context, code_size, predict_terms)

    y_input = keras.layers.Input((predict_terms, spect_height, n_lags))
    y_encoded = keras.layers.TimeDistributed(encoder_model)(y_input)

    # Loss
    dot_product_probs = CPCLayer()([preds, y_encoded])

    # Model
    cpc_model = keras.models.Model(inputs=[x_input, y_input], outputs=dot_product_probs)

    # Compile model
    cpc_model.compile(
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    cpc_model.summary()

    return cpc_model

In [50]:
# memmap ver
def train_model(data, x_data, y_data, data_vis, x_data_vis, y_data_vis, n_lags, spect_height, epochs, batch_size, output_dir, code_size, lr=2e-4, terms=3, predict_terms=3, color=True):

    # Prepare data
    # clean up generator more to work by subset
    train_data = Shuffled_memmap_subset(
        data=data,
        x_data=x_data,
        y_data=y_data,
        subset='train', 
        n_lags=n_lags,
        spect_height=spect_height
        )

    test_data = Shuffled_memmap_subset(
        data=data_vis,
        x_data=x_data_vis,
        y_data=y_data_vis,
        subset='test', 
        n_lags=n_lags,
        spect_height=spect_height
        )
    
    # Prepares the model
    model = network_cpc(spect_height=spect_height, n_lags=n_lags, terms=terms, predict_terms=predict_terms,
                        code_size=code_size, input_shape=(spect_height, n_lags), learning_rate=lr)

    # Callbacks
    history = keras.callbacks.History()
    reduce_LR_Plateau = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=1/3, 
        patience=2, 
        min_lr=1e-4)
    #callbacks = [history, reduce_LR_Plateau]
    callbacks = [history]
    
    # Trains the model
    modeled = model.fit_generator(
        generator=train_data.data_iterator(),
        steps_per_epoch=len(train_data),
        validation_data=test_data.data_iterator(),
        validation_steps=len(test_data),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks
    )

    # Saves the model
    # Remember to add custom_objects={'CPCLayer': CPCLayer} to load_model when loading from disk
    model.save(join(output_dir, 'mem_cpc.h5'))

    # Saves the encoder alone
    encoder = model.layers[1].layer
    encoder.save(join(output_dir, 'mem_encoder.h5'))
    
    # plotting loss
    train_loss = modeled.history['loss']
    val_loss = modeled.history['val_loss']
    epoch_count = range(1, epochs+1)    # 10 epochs

    plt.plot(epoch_count, train_loss, 'r--')
    plt.plot(epoch_count, val_loss, 'o-')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.title('')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    t = time.localtime()
    timestamp = time.strftime('%b-%d-%Y_%H%M', t)
    plt.savefig(output_dir + "img/" + timestamp + "_mem.png")

    return model

In [80]:
train_model(
    # x/y train data
    data=data,
    x_data=x_data,
    y_data=y_data,
    # x/y test data
    data_vis=data_vis,
    x_data_vis=x_data_vis,
    y_data_vis=y_data_vis,
    
    # params
    n_lags=2, 
    spect_height=32,
    epochs=10,
    batch_size=8,
    output_dir='models/memmap',
    code_size=512,
    lr=2e-4,
    terms=3,
    predict_terms=3,
    color=True
)

(?, 32, 2)
conv1d (?, 31, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 2)             0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 31, 2)             10        
_________________________________________________________________
batch_normalization_v1_10 (B (None, 31, 2)             8         
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 31, 2)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 30, 2)             10        
_________________________________________________________________
batch_normalization_v1_11 (B (None, 30, 2)             8         
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 30, 2)     

TypeError: only integer scalar arrays can be converted to a scalar index